<a href="https://colab.research.google.com/github/sayantika21175/NLP/blob/main/Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2b448836f3151190de62d9c9dd20bc69a90bc9656001822eb0b479f727b8e163
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [7]:
import torch
from datasets import load_dataset

full_dataset = load_dataset("cnn_dailymail", name="3.0.0")

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [8]:
sample_size=100
sample_data=full_dataset["train"].filter(lambda r: 'CNN' in r["article"][:25]).shuffle(seed=42).select(range(sample_size))

Filter:   0%|          | 0/287113 [00:00<?, ? examples/s]

In [9]:
sample_data

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 100
})

In [11]:
display(sample_data.to_pandas())

,article,highlights,id
0,(CNN) -- A magnitude 6.7 earthquake rattled Pa...,Papua New Guinea is on the so-called Ring of F...,8093dba7bc2260c26f18939826909ef27549c758
1,(CNN) -- Pakistan took big steps towards level...,Australia collapse to 88 all out on opening da...,67d626156f971d0bf55e5f2a48e1ed965eb622a6
2,(CNN) -- Federal prosecutors are pushing to fo...,Jared Loughner is refusing the government's re...,0d02fb8f0d406db956b128a5c1cc7bf3f13860a6
3,"Centennial, Colorado (CNN) -- McKayla Hicks sa...",Shooting victim McKayla Hicks went to hearing ...,39aee887c6d34bd311c826142b14037e6f2639ee
4,(CNN) -- Double-amputee sprinter Oscar Pistori...,Oscar Pistorius to become first double-amputee...,cc83ecdf08f0b598c3b97b3e2819c7e0ae7ca4f2
...,...,...,...
95,(CNN) -- Samuel Eto'o netted a superb hat-tric...,Samuel Eto'o scored a hat-trick as Inter Milan...,6c1924f5852b6980a0835877d3f9591a00c70f37
96,Washington (CNN) -- President Barack Obama's r...,Obama raised almost $30 million less than Romn...,0a5691b8fe654b6b2cdace5ab87aff2ee4c23577
97,(CNN) -- Violence swept across Syria on Friday...,NEW: U.N. Secretary-General Ban Ki-moon joins ...,2cc6e4db9758192ac467bbd7424782e4c92206c1
98,(CNN) -- New HIV infections have fallen worldw...,New infections in sub-Saharan Africa 15 percen...,acb2148184f83ecb516ad19a1b0a0e1bc5047237


In [13]:
import pandas as pd
import gc
from transformers import AutoTokenizer, T5ForConditionalGeneration
from rouge_score import rouge_scorer

In [15]:
def batch_generator(data:list,batch_size:int):
  """creates batches of size 'batch_size' from a list"""
  s = 0
  e = s+batch_size
  while (s< len(data)):
    yield data[s:e]
    s=e
    e= min(s+batch_size,len(data))

def summaries_with_t5(model_checkpoint:str,articles:list,batch_size:int=8)->list:
  """Compute summaries using t5 model. This is similar to a pipeline for t5 model but does tokenization manually.
  """
  if torch.cuda.is_available():
    device="cuda:0"
  else:
    device= "cpu"

  model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,model_max_length=1024)

  def perform_imference(batch:list)->list:
    inputs=tokenizer(batch,max_length=1024,padding=True,truncation=True,return_tensors="pt")
    summary_ids= model.generate(inputs.input_ids.to(device),attention_mask=inputs.attention_mask.to(device),num_beams=2,min_length=0,max_length=40)
    return tokenizer.batch_decode(summary_ids,skip_special_tokens=True)

  summaries=[]
  summary_articles=list(map(lambda article:"summarize: "+article,articles))
  for batch in batch_generator(summary_articles,batch_size):
    summaries+=perform_imference(batch)
    torch.cuda.empty_cache()
    gc.collect()
  del tokenizer
  del model
  torch.cuda.empty_cache()
  gc.collect()
  return summaries



In [16]:
t5_small_summaries= summaries_with_t5("t5-small",sample_data["article"])

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [18]:
reference_summaries= sample_data["highlights"]

In [19]:
display(pd.DataFrame({"reference":reference_summaries,"generated_summaries":t5_small_summaries}))

,reference,generated_summaries
0,Papua New Guinea is on the so-called Ring of F...,a magnitude 6.7 earthquake rattles Papua new G...
1,Australia collapse to 88 all out on opening da...,the two-Test cricket series is being played in...
2,Jared Loughner is refusing the government's re...,federal prosecutors want jared Lee Loughner to...
3,Shooting victim McKayla Hicks went to hearing ...,"new: ""he tried to kill people,"" a 17-year-old ..."
4,Oscar Pistorius to become first double-amputee...,double-amputee sprinter Oscar Pistorius will c...
...,...,...
95,Samuel Eto'o scored a hat-trick as Inter Milan...,holders Inter Milan thrash Werder Bremen 4-0 i...
96,Obama raised almost $30 million less than Romn...,president's re-election campaign raises $71 mi...
97,NEW: U.N. Secretary-General Ban Ki-moon joins ...,"at least 75 people were killed in protests, an..."
98,New infections in sub-Saharan Africa 15 percen...,new infections have fallen by 17 percent in th...


#Accuracy check

In [20]:
accuracy =0.0
for i in range(len(reference_summaries)):
  if(reference_summaries[i]==t5_small_summaries[i]):
    accuracy+=1
print(f"accuracy is {accuracy/len(reference_summaries)}")

accuracy is 0.0


In [23]:
#!pip install evaluate

## Rough score check

In [29]:
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [30]:
rough_score=evaluate.load("rouge")

In [33]:
def compute_rouge_score(generated:list,reference:list)->dict:
  """compute a ROUGE score on a batch of articles.
  This is a convenience function wrapping Hugging face 'rouge_score',
  which expects sentences to be seperated by newlines.
  """

  generated_with_newlines=[
      "\n".join(sent_tokenize(s.strip())) for s in generated
  ]

  reference_with_newlines=["\n".join(sent_tokenize(s.strip())) for s in reference]

  return rough_score.compute(predictions=generated_with_newlines,references=reference_with_newlines,use_stemmer=True)

In [34]:
compute_rouge_score(t5_small_summaries,reference_summaries)

{'rouge1': np.float64(0.30950750496548196),
 'rouge2': np.float64(0.10562688655910907),
 'rougeL': np.float64(0.22163709589066227),
 'rougeLsum': np.float64(0.2812675635506858)}

#if we fail to predict anything

In [35]:
compute_rouge_score(generated=["" for _ in range(len(reference_summaries))],reference=reference_summaries)

{'rouge1': np.float64(0.0),
 'rouge2': np.float64(0.0),
 'rougeL': np.float64(0.0),
 'rougeLsum': np.float64(0.0)}

#use stemmer to ignore the small differences

In [37]:
rough_score.compute(predictions=["Large language models beat world record"],
                    references=["Large language models beating world records"],
                    use_stemmer=False)


{'rouge1': np.float64(0.6666666666666666),
 'rouge2': np.float64(0.4000000000000001),
 'rougeL': np.float64(0.6666666666666666),
 'rougeLsum': np.float64(0.6666666666666666)}

In [38]:
rough_score.compute(predictions=["Large language models beat world record"],
                    references=["Large language models beating world records"],
                    use_stemmer=True)


{'rouge1': np.float64(1.0),
 'rouge2': np.float64(1.0),
 'rougeL': np.float64(1.0),
 'rougeLsum': np.float64(1.0)}

In [39]:
rough_score.compute(predictions=["Large language models beat world record"],
                    references=["Large"],
                    use_stemmer=True)

{'rouge1': np.float64(0.2857142857142857),
 'rouge2': np.float64(0.0),
 'rougeL': np.float64(0.2857142857142857),
 'rougeLsum': np.float64(0.2857142857142857)}

In [40]:
rough_score.compute(predictions=["Large"],
                    references=["Large language models beat world record"],
                    use_stemmer=True)

{'rouge1': np.float64(0.2857142857142857),
 'rouge2': np.float64(0.0),
 'rougeL': np.float64(0.2857142857142857),
 'rougeLsum': np.float64(0.2857142857142857)}

In [41]:
rough_score.compute(predictions=["Large Language"],
                    references=["Large language models beat world record"],
                    use_stemmer=True)

{'rouge1': np.float64(0.5),
 'rouge2': np.float64(0.33333333333333337),
 'rougeL': np.float64(0.5),
 'rougeLsum': np.float64(0.5)}

#Rouge1 is misleading because it checks the individual word not the meaning

In [43]:
rough_score.compute(predictions=["Models beat large language world record"],
                    references=["Large language models beat world record"],
                    use_stemmer=True)

{'rouge1': np.float64(1.0),
 'rouge2': np.float64(0.6),
 'rougeL': np.float64(0.6666666666666666),
 'rougeLsum': np.float64(0.6666666666666666)}